In [31]:
import pandas as pd
import os
import cv2
from os import listdir
import numpy as np
import struct
from scipy.io import wavfile as wav
import matplotlib.pyplot as plt
import IPython.display as ipd
from keras.models import Sequential
from tensorflow import keras
from keras.layers import Dense

## Static variables

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Ilyes/Desktop/melspectrogrammetadata/UrbanSound8k.csv'

## Create refined data for AI training

In [33]:
def refine_sound(path):
    try:
        rate, wav_sample = wav.read(path)
        wave_file = open(path,"rb")
        riff_fmt = wave_file.read(36)
        bit_depth_string = riff_fmt[-2:]
        bit_depth = struct.unpack("H",bit_depth_string)[0]
        return [rate,
            bit_depth,
            wav_sample.shape[1],
            wav_sample.shape[0]/rate,
            len(wav_sample)]
    except Exception:
        return None

def get_label_name(id_class):
    try:
        switcher = {
            "0" : "air_conditioner",
            "1" : "car_horn",
            "2" : "children_playing",
            "3" : "dog_bark",
            "4" : "drilling",
            "5" : "engine_idling",
            "6" : "gun_shot",
            "7" : "jackhammer",
            "8" :  "siren",
            "9" : "street_music",
        }
        return switcher.get(id_class, "nothing")
    except Exception:
        return None
    
    
def get_label(filename):
    try:
        id_class = filename.split("-",4)[1]
        return int(id_class)
    except Exception:
        return None

def get_fold_sounds(folder, data, labels):
    for file in listdir(folder):
        label = get_label(file)
        if label != None:
            sound_data = refine_sound(folder + file)
            if sound_data != None:
                data.append(sound_data)
                labels.append(label)
    return data, labels

def load_data(data_dir):
    X = []
    Y = []
    for img in os.listdir(data_dir):
            img_arr = cv2.imread(os.path.join(data_dir, img), cv2.IMREAD_COLOR)
            resized_arr = cv2.resize(img_arr, (500, 500)) 
            X.append(resized_arr)    
            Y.append(get_label(img))
    return np.array(X), Y

def get_all_sounds(folders):
    data = []
    labels = []
    for i in folders:
        data, labels = get_fold_sounds(urban_sound_folder + audio_folder + "fold" + str(i) + '/', data, labels)
    return data, labels

In [36]:

save_url = "C:/Users/Ilyes/Desktop/melspectrogram/fold1/"
# X is the training data and y the value of the data
x, y = load_data(save_url)

X = x.astype("float32") / 255
Y = keras.utils.to_categorical(y, 10)

print("Size: ", len(X))
print("Size: ", len(y))
input_dim = len(X[0])

MemoryError: Unable to allocate 2.44 GiB for an array with shape (873, 500, 500, 3) and data type float32

In [35]:
print(X[0])
print(y[0])

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
3


## AI training

In [28]:
model = Sequential()
model.add(Dense(12, input_dim=input_dim, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
history = model.fit(X, y, epochs=150, batch_size=10)
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'int'>"})

## Test a sound

In [ ]:
def path_class(filename):
    excerpt = us8k_data[us8k_data['slice_file_name'] == filename]

    path_name = os.path.join(urban_sound_folder + audio_folder, 'fold'+str(excerpt.fold.values[0]), filename)
    return path_name, excerpt['classID'].values[0]

In [ ]:
file_to_test = "69661-3-0-29.wav"
path, expected_label = path_class(file_to_test)
to_predict = [refine_sound(path)]

print("Prediction:", model.predict(to_predict))
print("Expected:", expected_label)

In [ ]:
predictions = (model.predict(X) > 0.5).astype(int)
# summarize the first 5 cases
for i in range(5):
	print('%s => %d (expected %d)' % (X[i], predictions[i], y[i]))

In [ ]:
#pd.set_option('display.max_columns', None)
#
#appended = []
#for i in range(1, 11):
#    appended.append(us8k_data[us8k_data.fold == i]['class'].value_counts())
#
#class_distribution = pd.DataFrame(appended)
#class_distribution = class_distribution.reset_index()
#class_distribution['index'] = ["fold" + str(x) for x in range(1, 11)]
#path, label = path_class(file_to_test)
#
#wav_plotter(path, label)

0 = air_conditioner
1 = car_horn
2 = children_playing
3 = dog_bark
4 = drilling
5 = engine_idling
6 = gun_shot
7 = jackhammer
8 = siren
9 = street_music
